In [2]:
import os
import numpy as np
import cv2
import pandas as pd
import re

In [9]:
treatments = ["Alt2", "Alt4", "Alt8", "Alt16"]
base_dir = "/mnt/d/Flora project/Mask_Test2"
frames_dir = os.path.join(base_dir, "Frames")
masks_dir = os.path.join(base_dir, "Masks")
clip_names = os.listdir(frames_dir) 

In [8]:
# Initialize dictionary to store masks
masks_dict = {}

# Loop through each treatment
for treatment in treatments:
    treatment_path = os.path.join(masks_dir, treatment)
    masks_dict[treatment] = {}  # Create a sub-dictionary for this treatment

    # Loop through each clip folder in the treatment directory
    for clip_name in os.listdir(treatment_path):
        clip_path = os.path.join(treatment_path, clip_name)

        if os.path.isdir(clip_path):  # Ensure it's a directory
            # Loop through each CSV file in the clip folder
            for filename in os.listdir(clip_path):
                if filename.endswith(".csv"):  # Ensure it's a CSV file
                    file_path = os.path.join(clip_path, filename)
                    
                    # Load the mask (using numpy or pandas)
                    mask = np.loadtxt(file_path, delimiter=",")  # Use pd.read_csv(file_path) if needed
                    
                    # Store in dictionary
                    masks_dict[treatment][filename] = mask

# Now, masks_dict contains:
# masks_dict["unaltered_mask"]["some_file.csv"] -> Corresponding mask
# masks_dict["Alt2"]["another_file.csv"] -> Corresponding mask

print("Masks dictionary successfully created!")

Masks dictionary successfully created!


In [25]:
# treatments = ["unaltered_mask", "plain", "Alt2", "Alt4", "Alt8", "Alt16"]
treatment = "Alt16"

base_dir = "/mnt/d/Flora project/Mask_Test2"
frame_dir = os.path.join(base_dir, "Frames/Botia_lohachata_1_C_L_3_L")
mask_dir = os.path.join(base_dir, "Masks", f"Altered_Masks/Botia_lohachata_1_C_L_3_L/{treatment}")
# frame_dir = "/mnt/d/Flora project/Mask_Test/Frames/Botia_lohachata_1_C_6"
# mask_dir = "/mnt/d/Flora project/Mask_Test/Basic_Masks/Botia_lohachata_1_C_6"
# mask_dir = f"/mnt/d/Flora project/Mask_Test/Altered_masks3/Botia_lohachata_1_C_6/{treatment}"
output_dir = "/mnt/d/Flora project/Mask_Test2/Treatment_Output/Botia_lohachata_1_C_L_3_L"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

In [14]:
def extract_frame_number(filename):
    match = re.search(r'frame_(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

In [15]:
frames = sorted(os.listdir(frame_dir))
masks = sorted(os.listdir(mask_dir), key=extract_frame_number)  

sample_frame = cv2.imread(os.path.join(frame_dir, frames[0]))
height, width, _ = sample_frame.shape
fps = 60

In [26]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# stripe_video1 = cv2.VideoWriter(os.path.join(output_dir, f"{treatment}_unaltered_mask.mp4"), fourcc, fps, (width, height))
# stripe_video2 = cv2.VideoWriter(os.path.join(output_dir, f"{treatment}_fish.mp4"), fourcc, fps, (width, height))
# stripe_video3 = cv2.VideoWriter(os.path.join(output_dir, f"{treatment}_background.mp4"), fourcc, fps, (width, height))
stripe_video4 = cv2.VideoWriter(os.path.join(output_dir, f"{treatment}_both.mp4"), fourcc, fps, (width, height))

In [27]:
########## Stripey fish with mean_luminance background

for frame_name, mask_name in zip(frames, masks):
    frame_path = os.path.join(frame_dir, frame_name)
    mask_path = os.path.join(mask_dir, mask_name)

    # Read images
    frame = cv2.imread(frame_path)
    mask = pd.read_csv(mask_path, header=None).values  # Convert CSV to NumPy array

    # Compute mean luminance for the background and the fish
    background_pixels = frame[mask == 0]
    fish_pixels = frame[(mask == 1) | (mask == 2)]

    mean_background = (
        np.mean(background_pixels, axis=0).astype(np.uint8)
        if len(background_pixels) > 0
        else np.array([0, 0, 0], dtype=np.uint8)
    )
    mean_fish = (
        np.mean(fish_pixels, axis=0).astype(np.uint8)
        if len(fish_pixels) > 0
        else np.array([0, 0, 0], dtype=np.uint8)
    )


    # Create mean luminance images
    mean_fish_img = np.full(frame.shape, mean_fish, dtype=np.uint8)
    mean_background_img = np.full(frame.shape, mean_background, dtype=np.uint8)

    # Apply treatments
    unaltered_mask = frame.copy()
    mean_fish_treatment = frame.copy()
    mean_background_treatment = frame.copy()
    mean_both_treatment = frame.copy()

    # Apply modifications
    mean_background_treatment[mask == 0] = mean_background  # Background with mean luminance
    mean_both_treatment[mask == 0] = mean_background  # Background with mean luminance

    # Modify fish pixels based on mask values
    mean_fish_treatment[mask == 1] = [255, 255, 255]  # White where mask == 1
    mean_fish_treatment[mask == 2] = [0, 0, 0]  # Black where mask == 2

    mean_both_treatment[mask == 1] = [255, 255, 255]  # White where mask == 1
    mean_both_treatment[mask == 2] = [0, 0, 0]  # Black where mask == 2

    # Write frames to videos
    # stripe_video1.write(unaltered_mask)
    # stripe_video2.write(mean_fish_treatment)
    # stripe_video3.write(mean_background_treatment)
    stripe_video4.write(mean_both_treatment)

# Release video writers
#stripe_video1.release()
#stripe_video2.release()
# stripe_video3.release()
stripe_video4.release()


In [43]:
#### Saving as folder of frames and video



# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
stripe_video3 = cv2.VideoWriter(os.path.join(output_dir, f"{treatment}_background.mp4"), fourcc, fps, (width, height))
stripe_video4 = cv2.VideoWriter(os.path.join(output_dir, f"{treatment}_both.mp4"), fourcc, fps, (width, height))

# Create directories for saving PNG frames
png_output_dir3 = os.path.join(output_dir, f"{treatment}_background_frames")
png_output_dir4 = os.path.join(output_dir, f"{treatment}_both_frames")
os.makedirs(png_output_dir3, exist_ok=True)
os.makedirs(png_output_dir4, exist_ok=True)

########## Process frames
for i, (frame_name, mask_name) in enumerate(zip(frames, masks)):
    frame_path = os.path.join(frame_dir, frame_name)
    mask_path = os.path.join(mask_dir, mask_name)

    # Read images
    frame = cv2.imread(frame_path)
    mask = pd.read_csv(mask_path, header=None).values  # Convert CSV to NumPy array

    # Compute mean luminance for the background
    background_pixels = frame[mask == 0]
    mean_background = (
        np.mean(background_pixels, axis=0).astype(np.uint8)
        if len(background_pixels) > 0
        else np.array([0, 0, 0], dtype=np.uint8)
    )

    # Create modified treatments
    mean_background_treatment = frame.copy()
    mean_both_treatment = frame.copy()

    # Apply modifications
    mean_background_treatment[mask == 0] = mean_background  # Background with mean luminance
    mean_both_treatment[mask == 0] = mean_background  # Background with mean luminance

    # Modify fish pixels based on mask values
    mean_both_treatment[mask == 1] = [255, 255, 255]  # White where mask == 1
    mean_both_treatment[mask == 2] = [0, 0, 0]  # Black where mask == 2

    # Write frames to videos
    stripe_video3.write(mean_background_treatment)
    stripe_video4.write(mean_both_treatment)

    # Save frames as PNG images
    frame_filename = f"{i:04d}.png"
    cv2.imwrite(os.path.join(png_output_dir3, frame_filename), mean_background_treatment)
    cv2.imwrite(os.path.join(png_output_dir4, frame_filename), mean_both_treatment)

# Release video writers
stripe_video3.release()
stripe_video4.release()

print(f"Frames saved as PNG in {png_output_dir3} and {png_output_dir4}")
print(f"Videos saved in {output_dir}")


Frames saved as PNG in /mnt/d/Flora project/Mask_Test/Treatment_Output3/Botia_lohachata_1_C_6/Alt16_background_frames and /mnt/d/Flora project/Mask_Test/Treatment_Output3/Botia_lohachata_1_C_6/Alt16_both_frames
Videos saved in /mnt/d/Flora project/Mask_Test/Treatment_Output3/Botia_lohachata_1_C_6


In [ ]:
########## trying to get it to apply all the treatments

# for each clip I want to produce videos and folders of frames for "unaltered", "plain", "Alt2", "Alt4", "Alt8", "Alt16"
# apart from "unaltered" they all need to have mean lumninance background. 
# for unaltered and plain it needs to load in the 'basic_mask' as mask_dir but for the other 3 'altered' treatments it needs to load in a different mask_dir
## # mask_dir = "/mnt/d/Flora project/Mask_Test/Basic_Masks/clip_name"
## # mask_dir = f"/mnt/d/Flora project/Mask_Test/Altered_masks/clip_name/{treatment}"

for frame_name, mask_name in zip(frames, masks):
    frame_path = os.path.join(frame_dir, frame_name)
    mask_path = os.path.join(mask_dir, mask_name)

    # Read images
    frame = cv2.imread(frame_path)
    mask = pd.read_csv(mask_path, header=None).values  # Convert CSV to NumPy array

    ## IF "unaltered" or 
    # "plain"
    # Compute mean luminance for the background and the fish
    background_pixels = frame[mask == 0]
    fish_pixels = frame[(mask == 1) | (mask == 2)]

    mean_background = (
        np.mean(background_pixels, axis=0).astype(np.uint8)
        if len(background_pixels) > 0
        else np.array([0, 0, 0], dtype=np.uint8)
    )
    mean_fish = (
        np.mean(fish_pixels, axis=0).astype(np.uint8)
        if len(fish_pixels) > 0
        else np.array([0, 0, 0], dtype=np.uint8)
    )


    # Create mean luminance images
    mean_fish_img = np.full(frame.shape, mean_fish, dtype=np.uint8)
    mean_background_img = np.full(frame.shape, mean_background, dtype=np.uint8)

    # Apply treatments
    unaltered_mask = frame.copy()
    mean_fish_treatment = frame.copy()
    mean_background_treatment = frame.copy()
    mean_both_treatment = frame.copy()

    # Apply modifications
    mean_background_treatment[mask == 0] = mean_background  # Background with mean luminance
    mean_both_treatment[mask == 0] = mean_background  # Background with mean luminance

    # Modify fish pixels based on mask values
    mean_fish_treatment[mask == 1] = [255, 255, 255]  # White where mask == 1
    mean_fish_treatment[mask == 2] = [0, 0, 0]  # Black where mask == 2

    mean_both_treatment[mask == 1] = [255, 255, 255]  # White where mask == 1
    mean_both_treatment[mask == 2] = [0, 0, 0]  # Black where mask == 2

    # Write frames to videos
    # stripe_video1.write(unaltered_mask)
    # stripe_video2.write(mean_fish_treatment)
    stripe_video3.write(mean_background_treatment)
    stripe_video4.write(mean_both_treatment)

# Release video writers
#stripe_video1.release()
#stripe_video2.release()
stripe_video3.release()
stripe_video4.release()
